In [44]:
#Using subprocess and sextractor to make measurements in fits images.
from subprocess import call, check_output, check_call
import astropy.io.fits as pf
import numpy as np

In [45]:
def gauss(size):                                            #Define a gaussian for creating an artificial source of sigma=size.
    size = int(size)
    x, y = np.mgrid[-25:25, -25:25]                         #Create a 50x50 grid array
    g = np.exp(-(x**2/float(size)+y**2/float(size)))        #Generate a Gaussian array
    return g / g.sum()                                      #Normalize

In [46]:
noise = np.random.poisson(100,size=(50,50))                 #Background noise  
signal = 3000*gauss(10)                                     #Artificial Source
img = signal+noise                                          #Pretend image

In [47]:
test = 'test_img.fits'                                      #Image name
segmap = 'test_segmap.fits'                                 #Segmentation map name
cat = 'test.cat'                                            #Sextractor catalog name
pf.writeto(test,img,overwrite=True)                           #Write image to a fits file

In [48]:
config = check_output('sextractor -d',shell=True)           #Save shell output from sextractor default config file into string
print config                                                #print string

# Default configuration file for SExtractor 2.19.5
# EB 2013-12-14
#
 
#-------------------------------- Catalog ------------------------------------
 
CATALOG_NAME     test.cat       # name of the output catalog
CATALOG_TYPE     ASCII_HEAD     # NONE,ASCII,ASCII_HEAD, ASCII_SKYCAT,
                                # ASCII_VOTABLE, FITS_1.0 or FITS_LDAC
PARAMETERS_NAME  default.param  # name of the file containing catalog contents
 
#------------------------------- Extraction ----------------------------------
 
DETECT_TYPE      CCD            # CCD (linear) or PHOTO (with gamma correction)
DETECT_MINAREA   5              # min. # of pixels above threshold
DETECT_THRESH    1.5            # <sigmas> or <threshold>,<ZP> in mag.arcsec-2
ANALYSIS_THRESH  1.5            # <sigmas> or <threshold>,<ZP> in mag.arcsec-2
 
FILTER           Y              # apply filter for detection (Y or N)?
FILTER_NAME      default.conv   # name of the file containing the filter
 
DEBLEND_NTHRESH  32           

In [49]:
with open("default.sex", "w") as txt:                       #Save sextractor config file to default name
    txt.write(config)

In [50]:
#Run sextractor on test saving catalog of measurements listed in default.param to cat and adjusting parameters in config file on the command line
call(["sextractor", test, "-CATALOG_NAME", cat, "-PARAMETERS_NAME", "default.param", "-DETECT_MINAREA", "5", "-DETECT_THRESH", "1.5", "-ANALYSIS_THRESH", "0.8", "-DEBLEND_NTHRESH", "64","-DEBLEND_MINCONT","0.0005", "-MAG_ZEROPOINT", "0", "-BACK_SIZE", "32", "-BACK_FILTERSIZE", "3", "-PIXEL_SCALE", "0.09", "-BACK_VALUE", "0", "-CHECKIMAGE_TYPE", "SEGMENTATION", "-CHECKIMAGE_NAME", segmap, "-VERBOSE_TYPE", "QUIET"])

0

In [51]:
param = np.genfromtxt('default.param',dtype='a20')          #Open sextractor parameter file as array of strings
cat = np.genfromtxt(cat,dtype=float,names=','.join(param))  #Open sextractor catalog in array with names from parameter file (One could save the catalog as a fits_1.0 file by adding "-CATALOG_TYPE", "FITS_1.0" to the argument list in the call argument list)
print cat['FLUX_AUTO']/cat['FLUXERR_AUTO']                  #Print the signal to noise ratio of your detection

25.3197685478


In [52]:
call(['ds9',test,segmap])                                   #Open your image and segmentation map in ds9

0